## GPU

在1.5节中，我们回顾了过去20年计算能⼒的快速增⻓。简⽽⾔之，⾃2000年以来，GPU性能每⼗年增⻓1000倍。
本节，我们将讨论如何利⽤这种计算性能进⾏研究。⾸先是如何使⽤单个GPU，然后是如何使⽤多个GPU和
多个服务器（具有多个GPU）。

In [1]:
# 我们先看看如何使⽤单个NVIDIA GPU进⾏计算。⾸先，确保⾄少安装了⼀个NVIDIA GPU。然后，下
# 载NVIDIA驱动和CUDA 80 并按照提⽰设置适当的路径。当这些准备⼯作完成，就可以使⽤nvidia-smi命令
# 来查看显卡信息。
!nvidia-smi

Wed Jul 26 07:09:46 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.23                 Driver Version: 536.23       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   36C    P3              20W /  55W |      0MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

在PyTorch中，每个数组都有⼀个设备（device），我们通常将其称为环境（context）。默认情况下，所有
变量和相关的计算都分配给CPU。有时环境可能是GPU。当我们跨多个服务器部署作业时，事情会变得更加
棘⼿。通过智能地将数组分配给环境，我们可以最⼤限度地减少在设备之间传输数据的时间。例如，当在带
有GPU的服务器上训练神经⽹络时，我们通常希望模型的参数在GPU上。

要运⾏此部分中的程序，⾄少需要两个GPU。注意，对⼤多数桌⾯计算机来说，这可能是奢侈的，但在云中
很容易获得。例如可以使⽤AWS EC2的多GPU实例。本书的其他章节⼤都不需要多个GPU，⽽本节只是为了
展⽰数据如何在不同的设备之间传递。

### 计算设备

我们可以指定⽤于存储和计算的设备，如CPU和GPU。默认情况下，张量是在内存中创建的，然后使⽤CPU计
算它。

In [2]:
# 在PyTorch中，CPU和GPU可以⽤torch.device('cpu') 和torch.device('cuda')表⽰。应该注意的是，cpu设
# 备意味着所有物理CPU和内存，这意味着PyTorch的计算将尝试使⽤所有CPU核⼼。然⽽，gpu设备只代表⼀
# 个卡和相应的显存。如果有多个GPU，我们使⽤torch.device(f'cuda:{i}') 来表⽰第i块GPU（i从0开始）。
# 另外，cuda:0和cuda是等价的。

In [3]:
import torch
from torch import nn

In [4]:
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [5]:
# 我们可以查询可用gpu的数量
torch.cuda.device_count()

1

In [6]:
# 我们定义了两个方便的函数，这两个函数允许我们在不存在所需所有GPU的情况下运行代码

In [7]:
from d2l import torch as d2l

In [8]:
# d2l.try_gpu(i=0)
def try_gpu(i=0):
    """如果存在，则返回gpu(i), 否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

# d2l.try_all_gpus
def try_all_gpus():
    """返回所有可用的GPU,如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

In [9]:
try_gpu(), try_gpu(0), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cuda', index=0),
 [device(type='cuda', index=0)])

### 张量与GPU

In [10]:
# 我们可以查询张量所在的设备。默认情况下，张量是在CPU上创建的。

In [11]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

需要注意的是，⽆论何时我们要对多个项进⾏操作，它们都必须在同⼀个设备上。例如，如果我们对两个张
量求和，我们需要确保两个张量都位于同⼀个设备上，否则框架将不知道在哪⾥存储结果，甚⾄不知道在哪
⾥执⾏计算。

#### 存储在GPU上

In [12]:
# 有⼏种⽅法可以在GPU上存储张量。例如，我们可以在创建张量时指定存储设备。接下来，我们在第⼀个gpu上
# 创建张量变量X。在GPU上创建的张量只消耗这个GPU的显存。我们可以使⽤nvidia-smi命令查看显存使⽤情
# 况。⼀般来说，我们需要确保不创建超过GPU显存限制的数据。

In [13]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [15]:
# 假设我们至少有两个GPU，下面的代码将在第二个GPU上创建一个随机变量
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.5291, 0.4901, 0.8742],
        [0.5900, 0.3551, 0.7226]])

In [16]:
# 由于本电脑只有一个GPU，所以,Y是在CPU上
Y.device

device(type='cpu')

#### 复制

如果我们要计算X + Y，我们需要决定在哪⾥执⾏这个操作。例如，如 图5.6.1所⽰，我们可以将X传输到第⼆
个GPU并在那⾥执⾏操作。不要简单地X加上Y，因为这会导致异常，运⾏时引擎不知道该怎么做：它在同⼀
设备上找不到数据会导致失败。由于Y位于第⼆个GPU上，所以我们需要将X移到那⾥，然后才能执⾏相加运
算。

In [17]:
Z = Y.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[0.5291, 0.4901, 0.8742],
        [0.5900, 0.3551, 0.7226]], device='cuda:0')


In [19]:
# 现在数据在同一个GPU上，我们可以将它们相加
X + Z

tensor([[1.5291, 1.4901, 1.8742],
        [1.5900, 1.3551, 1.7226]], device='cuda:0')

In [21]:
# 假设变量Z已经存在于第二个GPU上。如果我们还是调用Z.cuda(1)会发生什么？它将返回Z，而不会复制并分
# 配新内存
Z.cuda(0) is Z

True

#### 旁注

⼈们使⽤GPU来进⾏机器学习，因为单个GPU相对运⾏速度快。但是在设备（CPU、GPU和其他机器）之间
传输数据⽐计算慢得多。这也使得并⾏化变得更加困难，因为我们必须等待数据被发送（或者接收），然后
才能继续进⾏更多的操作。这就是为什么拷⻉操作要格外⼩⼼。根据经验，多个⼩操作⽐⼀个⼤操作糟糕得
多。此外，⼀次执⾏⼏个操作⽐代码中散布的许多单个操作要好得多。如果⼀个设备必须等待另⼀个设备才
能执⾏其他操作，那么这样的操作可能会阻塞。这有点像排队订购咖啡，⽽不像通过电话预先订购：当客⼈
到店的时候，咖啡已经准备好了。

最后，当我们打印张量或将张量转换为NumPy格式时，如果数据不在内存中，框架会⾸先将其复制到内存中，
这会导致额外的传输开销。更糟糕的是，它现在受制于全局解释器锁，使得⼀切都得等待Python完成。

### 神经网络与GPU

In [22]:
# 类似地，神经网络模型可以指定设备。下面的代码将模型参数放在GPU上
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

在接下来的⼏章中，我们将看到更多关于如何在GPU上运⾏模型的例⼦，因为它们将变得更加计算密集。
当输⼊为GPU上的张量时，模型将在同⼀GPU上计算结果。

In [23]:
net(X)

tensor([[0.7778],
        [0.7778]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [24]:
# 让我们确认模型参数存储在同一个GPU上
net[0].weight.data.device

device(type='cuda', index=0)

总之，只要所有的数据和参数都在同⼀个设备上，我们就可以有效地学习模型。在下⾯的章节中，我们将看
到⼏个这样的例⼦。

⼩结

• 我们可以指定⽤于存储和计算的设备，例如CPU或GPU。默认情况下，数据在主内存中创建，然后使
⽤CPU进⾏计算。

• 深度学习框架要求计算的所有输⼊数据都在同⼀设备上，⽆论是CPU还是GPU。

• 不经意地移动数据可能会显著降低性能。⼀个典型的错误如下：计算GPU上每个⼩批量的损失，并在命令⾏中将其报告给⽤⼾（或将其记录在NumPyndarray中）时，将触发全局解释器锁，从⽽使所有GPU阻塞。最好是为GPU内部的⽇志分配内存，并且只移动较⼤的⽇志。

In [30]:
#  尝试⼀个计算量更⼤的任务，⽐如⼤矩阵的乘法，看看CPU和GPU之间的速度差异。再试⼀个计算量很
# ⼩的任务呢？

import time
# 定义矩阵大小
matrix_size = 10000

# 生成两个随机矩阵
matrix_a = torch.rand(matrix_size, matrix_size)
matrix_b = torch.rand(matrix_size, matrix_size)

# 比较在CPU和GPU上矩阵乘法的速度差异
# 使用CPU
start_time = time.time()
result_cpu = torch.matmul(matrix_a, matrix_b)
end_time = time.time()
time_cpu = end_time - start_time

# 使用GPU
device = torch.device('cuda:0')
matrix_a_gpu = matrix_a.to(device)
matrix_b_gpu = matrix_b.to(device)

start_time = time.time()
result_gpu = torch.matmul(matrix_a_gpu, matrix_b_gpu)
torch.cuda.synchronize() # 等待GPU计算完成
end_time = time.time()
time_gpu = end_time - start_time

print('CPU_time: %f' %time_cpu)
print('GPU_time: %f' %time_gpu)

CPU_time: 5.173856
GPU_time: 0.516229


In [31]:
# 现在我们来试一个计算量很小的任务
# 例如：元素级操作（Element-wise operation）

# 定义一个小张量
small_tensor = torch.ones(100, 100)

# 使用CPU计算
start_time = time.time()
result_cpu = small_tensor * 2
end_time = time.time()
time_cpu = end_time - start_time

# 使用GPU计算
small_tensor_gpu = small_tensor.to(device)
start_time = time.time()
result_gpu = small_tensor_gpu * 2
torch.cuda.synchronize() # 等待GPU计算完成
end_time = time.time()
time_gpu = end_time - start_time

print('CPU_time: %f' %time_cpu)
print('GPU_time: %f' %time_gpu)

CPU_time: 0.000000
GPU_time: 0.015627


In [32]:
torch.bmm??

In [35]:
#  测量计算1000个100×100矩阵的矩阵乘法所需的时间，并记录输出矩阵的Frobenius范数，⼀次记录⼀
# 个结果，⽽不是在GPU上保存⽇志并仅传输最终结果。

# 定义矩阵数量和矩阵大小
num_matrices = 1000
matrix_size = 100

# 生成随机矩阵
matrices_a = torch.rand(num_matrices, matrix_size, matrix_size)
matrices_b = torch.rand(num_matrices, matrix_size, matrix_size)

# 在GPU上执行矩阵乘法并测量时间
device = torch.device('cuda:0')

start_time = time.time()
matrices_result = torch.bmm(matrices_a.to(device), matrices_b.to(device))
torch.cuda.synchronize() # 等待GPU计算完成
end_time = time.time()
time_elapsed = end_time - start_time

# 计算并记录每个输出矩阵的Frobenius范数
frobenius_norms = []
for i in range(num_matrices):
    frobenius_norm = torch.norm(matrices_result[i], 'fro')
    frobenius_norms.append(frobenius_norm.item())

print('GPU_time: %f' %time_elapsed)
print('Frobenius范数：\n')
print(frobenius_norms[0])

GPU_time: 0.703619
Frobenius范数：

2507.86328125
